# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [1]:
import os

import btrack
import imageio
import napari

import numpy as np

In [2]:
PATH = "/media/quantumjot/Data/Test"

### method 1 - using a numpy array

In this example, each image from the timelapse is a 3D volume (32 x 1200 x 1200) and there are 11 timepoints

In [3]:
def segmentation_arr(path):
    """Segmentation as numpy array."""
    
    stack = []
    files = os.listdir(path)
    files.sort()
    
    for filename in files:
        print(filename)
        
        # note, change this to imageio.imread for 2D data
        img = imageio.volread(os.path.join(path, filename))
        stack.append(img)
        
    return np.stack(stack, axis=0)

In [4]:
stack = segmentation_arr(PATH)
stack.shape

labels_0.tif
labels_1.tif
labels_2.tif
labels_3.tif
labels_4.tif
labels_5.tif
labels_6.tif
labels_7.tif
labels_8.tif
labels_9.tif


(10, 32, 1200, 1200)

### Method 2 - using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format

In [5]:
def segmentation_generator(path):
    """Segmentation generator"""
    files = os.listdir(path)
    files.sort()
    
    for filename in files:
        
        # note, change this to imageio.imread for 2D data
        img = imageio.volread(os.path.join(path, filename))
        
        yield img

In [6]:
generator = segmentation_generator(PATH)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 10x)

In [7]:
obj_from_arr = btrack.utils.segmentation_to_objects(stack, scale=(2., 1., 1.))

[INFO][2021/04/16 03:02:38 PM] Localizing objects from segmentation...
[INFO][2021/04/16 03:03:06 PM] Objects are of type: <class 'dict'>
[INFO][2021/04/16 03:03:06 PM] Found 11625 objects in 10 frames...


In [8]:
# inspect the first object
obj_from_arr[0]

,ID,x,y,z,t,dummy,states,label,prob
0,0,939.055033,20.643012,12.771904,0,False,0,5,0.0


In [9]:
obj_from_generator = btrack.utils.segmentation_to_objects(generator, scale=(2., 1., 1.))

[INFO][2021/04/16 03:03:07 PM] Localizing objects from segmentation...
[INFO][2021/04/16 03:03:35 PM] Objects are of type: <class 'dict'>
[INFO][2021/04/16 03:03:36 PM] Found 11625 objects in 10 frames...


In [10]:
# inspect the first object
obj_from_generator[0]

,ID,x,y,z,t,dummy,states,label,prob
0,0,939.055033,20.643012,12.771904,0,False,0,5,0.0


## run btrack with the objects

In [11]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    tracker.append(obj_from_generator)

    # set the volume
    tracker.volume=((0, 1200), (0, 1200), (-1e5, 64.))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

#     tracker.export('./test3.hdf5', obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=3)
    
    tracks = tracker.tracks

[INFO][2021/04/16 03:03:36 PM] Loaded btrack: /home/quantumjot/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/04/16 03:03:36 PM] btrack (v0.4.1) library imported
[INFO][2021/04/16 03:03:36 PM] Setting max XYZ search radius to: 100
[INFO][2021/04/16 03:03:36 PM] Starting BayesianTracker session
[INFO][2021/04/16 03:03:36 PM] Loading configuration file: ../models/cell_config.json
[INFO][2021/04/16 03:03:36 PM] Loading motion model: b'cell_motion'
[INFO][2021/04/16 03:03:36 PM] Setting max XYZ search radius to: 10
[INFO][2021/04/16 03:03:36 PM] Objects are of type: <class 'list'>
[INFO][2021/04/16 03:03:36 PM] Set volume to ((0, 1200), (0, 1200), (-100000.0, 64.0))
[INFO][2021/04/16 03:03:36 PM] Starting tracking... 
[INFO][2021/04/16 03:03:37 PM] Tracking objects in frames 0 to 10 (of 10)...
[INFO][2021/04/16 03:03:44 PM]  - Timing (Bayesian updates: 881.68ms, Linking: 8.44ms)
[INFO][2021/04/16 03:03:44 PM]  - Probabilities (Link: 1.00000, Lost: 0.50195)
[INFO][20

## visualize with napari

In [12]:
viewer = napari.Viewer()
viewer.add_labels(stack, scale=(1., 2., 1., 1.), name='Segmentation')
viewer.add_tracks(data, properties=properties, graph=graph, name='Tracks')

<Tracks layer 'Tracks' at 0x7f1ce0413a10>